## Evaluation of STT models

Load the benchmark dataset from huggingface

In [11]:
from datasets import load_dataset
import pandas as pd
dataset = load_dataset("openpecha/tibetan-voice-benchmark")
df = pd.DataFrame(dataset['test'])

In [13]:
df.head()

,file_name,uni,wylie,url,dept
0,STT_MV0105_0153_1036457_to_1046241.wav,ཁོང་གིས་སྐུ་ངོ་རྨུང་ལགས་རྩ་ནས་གྱོད་གཞིའི་ཞུ་ཚི...,khong gis sku ngo rmung lags rtsa nas gyod gzh...,https://d38pmlk0v88drf.cloudfront.net/mv_wav/S...,STT_MV
1,STT_MV1789_0365_2511796_to_2519156.wav,གཟུགས་རྡུལ་དབང་ཤེས་སྐྱེད་མིན་ཏེ། དེ་ནི་དབང་པོ་...,gzugs rdul dbang shes skyed min te/_de ni dban...,https://d38pmlk0v88drf.cloudfront.net/mv_wav/S...,STT_MV
2,STT_TT00079_00406.600-00407.400,དོན་དུ་གཉེར་བའི་བློ་འདི།,don du gnyer ba'i blo 'di/_,https://d38pmlk0v88drf.cloudfront.net/wav/STT_...,STT_TT
3,STT_TT00188_00312.400-00314.000,ཁ་སང་སྔན་དུ་བ། ག་གེ་ཞིག་ཡོད་པ་མ་རེད།,kha sang sngan du ba/_ga ge zhig yod pa ma red/_,https://d38pmlk0v88drf.cloudfront.net/wav/STT_...,STT_TT
4,STT_TT00078_00581.450-00582.300,གཙུག་ལག་ཁང་དེ་བ་ལ་ཡ་,gtsug lag khang de ba la ya _,https://d38pmlk0v88drf.cloudfront.net/wav/STT_...,STT_TT


In [14]:
df.shape

(30973, 5)

In [15]:
i = 2431
df.loc[i, 'url'], df.loc[i, 'uni']

('https://d38pmlk0v88drf.cloudfront.net/wav/STT_AB00051_0099_899260_to_907849.wav',
 'གནས་བརྟན་རྣམས་ལ་ཕྱག་བགྱིད་དགོངས་སུ་གསོལ། མཆོད་སྦྱིན་ཐ་མ་འདི་ནི་བཞེས་སུ་གསོལ། ཞེས་ཞུས་ལ་ཕུལ་ཅིག་ཅེས་གསུངས་ནས་བཏང་ངོ་། ')

### An Example inference from different models

In [35]:
from transformers import pipeline
import pyewts
generator = pipeline(model="openpecha/wav2vec2_run8")
opt = generator(df.loc[i, 'url'])
inf = opt['text']
print("openpecha/wav2vec2_run8", inf)

generator = pipeline(model="TenzinGayche/whisper-small-3")
opt = generator(df.loc[i, 'url'])
inf = opt['text']
converter = pyewts.pyewts()
print("TenzinGayche/whisper-small-3", converter.toUnicode(inf))

generator = pipeline(model="spsither/whipser-small-r2")
opt = generator(df.loc[i, 'url'])
inf = opt['text']
converter = pyewts.pyewts()
print("spsither/whipser-small-r2", converter.toUnicode(inf))

openpecha/wav2vec2_run8 གནས་བརྟན་རྣམས་ལ་ཕྱག་གིས་གོངས་སུ་གསོལ། མཆོད་སྦྱིན་མཐའ་མ་འདི་ནི་ཞེས་སུ་གསོལ། ཞེས་ཞུས་ལ་ཕུལ་གཅིག་ཅེས་གསུང་ནས་བཏང་ངོ་།
TenzinGayche/whisper-small-3 གནས་བརྟན་རྣམས་ལ་ཕྱག་གིས་དགོངས་སུ་གསོལ༎མཆོད་སྦྱིན་ཐ་མ་དེ་ནི་ཞེས་སུ་སྲོལ༎ཞེས་ཞུས་ལ་ཁུལ་གཅིག་ཅེས་གསུངས་ནས་བཏང་ངོ་༎
spsither/whipser-small-r2 ༎གནས་བརྟན་རྣམས་ལ་ཕྱག་གིས་དགོངས་སུ་གསོལ༎མཆོད་སྦྱིན་ཐ་མ་དེ་ནི་ཞེས་སུ་བསོལ་ཞུས་ལབུལ་ཅིག་ཅེས་གསུངས་ནས་བཏང་ངོ་༎


### Inference on all the benchmark data with the latest model

In [34]:
# import os
from transformers import pipeline
from datasets import Dataset
from tqdm.auto import tqdm

generator = pipeline(model="TenzinGayche/whisper-small-3", device="cuda:0")

def iterate_data(dataset):
    for i, item in enumerate(dataset):
        yield item["url"]

BATCH_SIZE = 16
predictions = []

# run streamed inference
for out in tqdm(generator(iterate_data(dataset['test']), batch_size=BATCH_SIZE), total=len(dataset['test'])//BATCH_SIZE):
    predictions.append(out["text"])

print(predictions[:5])

# df['inf']  = df.apply(lambda row: generator(row['url'])['text'], axis=1)
# updated_dataset = dataset.map(generate, batched=True, batch_size=8)

  0%|          | 0/1935 [00:00<?, ?it/s]

["khong gis sku ngo rmung lags rtsa nas gyod gzhi'i zhu rdzas gcig bdag zer gi red pe de'adras rtag par gnang gi re red khong gi tsha bos nag nyes bsags tang ma song na khos nang nye bza' ", "gzugs dus dbang shes kyis mi ldan//de ni dpon po las las//'dus pas de rnams skye byed na//tshogs pa de yang mi thos to//", "rnyed pa'i blo de ", 'kha sang sngon de ba ga ge cig yod ma red ', 'gtsug lag khang de ba la ya ']


In [37]:
df['inf'] = predictions

In [38]:
df.head()

,file_name,uni,wylie,url,dept,inf
0,STT_MV0105_0153_1036457_to_1046241.wav,ཁོང་གིས་སྐུ་ངོ་རྨུང་ལགས་རྩ་ནས་གྱོད་གཞིའི་ཞུ་ཚི...,khong gis sku ngo rmung lags rtsa nas gyod gzh...,https://d38pmlk0v88drf.cloudfront.net/mv_wav/S...,STT_MV,khong gis sku ngo rmung lags rtsa nas gyod gzh...
1,STT_MV1789_0365_2511796_to_2519156.wav,གཟུགས་རྡུལ་དབང་ཤེས་སྐྱེད་མིན་ཏེ། དེ་ནི་དབང་པོ་...,gzugs rdul dbang shes skyed min te/_de ni dban...,https://d38pmlk0v88drf.cloudfront.net/mv_wav/S...,STT_MV,gzugs dus dbang shes kyis mi ldan//de ni dpon ...
2,STT_TT00079_00406.600-00407.400,དོན་དུ་གཉེར་བའི་བློ་འདི།,don du gnyer ba'i blo 'di/_,https://d38pmlk0v88drf.cloudfront.net/wav/STT_...,STT_TT,rnyed pa'i blo de
3,STT_TT00188_00312.400-00314.000,ཁ་སང་སྔན་དུ་བ། ག་གེ་ཞིག་ཡོད་པ་མ་རེད།,kha sang sngan du ba/_ga ge zhig yod pa ma red/_,https://d38pmlk0v88drf.cloudfront.net/wav/STT_...,STT_TT,kha sang sngon de ba ga ge cig yod ma red
4,STT_TT00078_00581.450-00582.300,གཙུག་ལག་ཁང་དེ་བ་ལ་ཡ་,gtsug lag khang de ba la ya _,https://d38pmlk0v88drf.cloudfront.net/wav/STT_...,STT_TT,gtsug lag khang de ba la ya


In [39]:
df.to_csv('tibetan-voice-benchmark-with-inf.tsv', index=False, sep='\t')

In [ ]:
# import pyewts

# converter = pyewts.pyewts()

# df_out['inf_uni'] = df_out['inf'].map(lambda x : converter.toUnicode(x))

In [40]:
from evaluate import load

cer_metric = load("cer")

In [48]:
df_out = pd.read_csv('tibetan-voice-benchmark-with-inf.tsv')

cer = cer_metric.compute(references=[df_out.loc[0, 'wylie']], predictions=[df_out.loc[0, 'inf']])

print(cer)

0.325


In [49]:
df_out['cer'] = df_out.apply(lambda row: min(cer_metric.compute(references=[df_out.loc[0, 'wylie']], predictions=[df_out.loc[0, 'inf']]), 1), axis=1)

In [50]:
mean_cer = df_out['cer'].mean()

In [51]:
mean_cer

0.32500000000000007

In [52]:
mean_cer * 100

32.50000000000001

In [53]:
df_out.head()

,file_name,uni,wylie,url,dept,inf,cer
0,STT_MV0105_0153_1036457_to_1046241.wav,ཁོང་གིས་སྐུ་ངོ་རྨུང་ལགས་རྩ་ནས་གྱོད་གཞིའི་ཞུ་ཚི...,khong gis sku ngo rmung lags rtsa nas gyod gzh...,https://d38pmlk0v88drf.cloudfront.net/mv_wav/S...,STT_MV,khong gis sku ngo rmung lags rtsa nas gyod gzh...,0.325
1,STT_MV1789_0365_2511796_to_2519156.wav,གཟུགས་རྡུལ་དབང་ཤེས་སྐྱེད་མིན་ཏེ། དེ་ནི་དབང་པོ་...,gzugs rdul dbang shes skyed min te/_de ni dban...,https://d38pmlk0v88drf.cloudfront.net/mv_wav/S...,STT_MV,gzugs dus dbang shes kyis mi ldan//de ni dpon ...,0.325
2,STT_TT00079_00406.600-00407.400,དོན་དུ་གཉེར་བའི་བློ་འདི།,don du gnyer ba'i blo 'di/_,https://d38pmlk0v88drf.cloudfront.net/wav/STT_...,STT_TT,rnyed pa'i blo de,0.325
3,STT_TT00188_00312.400-00314.000,ཁ་སང་སྔན་དུ་བ། ག་གེ་ཞིག་ཡོད་པ་མ་རེད།,kha sang sngan du ba/_ga ge zhig yod pa ma red/_,https://d38pmlk0v88drf.cloudfront.net/wav/STT_...,STT_TT,kha sang sngon de ba ga ge cig yod ma red,0.325
4,STT_TT00078_00581.450-00582.300,གཙུག་ལག་ཁང་དེ་བ་ལ་ཡ་,gtsug lag khang de ba la ya _,https://d38pmlk0v88drf.cloudfront.net/wav/STT_...,STT_TT,gtsug lag khang de ba la ya,0.325


In [54]:

df_out.to_csv('tibetan-voice-benchmark-with-inf-cer.tsv', sep='\t', index=False)